# Use Case: Matched Instances Input


## Setup Imports

In [6]:
from auxiliary.nifti.io import read_nifti
from panoptica import MatchedInstancePair, Panoptic_Evaluator
import numpy as np

## Load Data

To demonstrate we use a reference and predicition of spine a segmentation with matched instances.


<img src="./spine_seg/matched_instance/fig.png" alt="matched_instance_figure" height="300"/>


In [7]:
ref_masks = read_nifti("./spine_seg/matched_instance/ref.nii.gz")
pred_masks = read_nifti("./spine_seg/matched_instance/pred.nii.gz")

# labels are matching
np.unique(ref_masks), np.unique(pred_masks)

(array([  0,   2,   3,   4,   5,   6,   7,   8,  26, 102, 103, 104, 105,
        106, 107, 108, 202, 203, 204, 205, 206, 207, 208], dtype=uint8),
 array([  0,   2,   3,   4,   5,   6,   7,   8,  26, 102, 103, 104, 105,
        106, 107, 108, 202, 203, 204, 205, 206, 207, 208], dtype=uint8))

## Run Evaluation

In [8]:
sample = MatchedInstancePair(prediction_arr=pred_masks, reference_arr=ref_masks)

evaluator = Panoptic_Evaluator(
    expected_input=MatchedInstancePair,
    iou_threshold=0.5,
)

result, debug_data = evaluator.evaluate(sample)
print(result)

evaluate took 0.059478044509887695 seconds to execute.
Number of instances in prediction: 22
Number of instances in reference: 22
True Positives (tp): 19
False Positives (fp): 3
False Negatives (fn): 3
Recognition Quality / F1 Score (RQ): 0.8636363636363636
Segmentation Quality (SQ): 0.8328184295330797 ± 0.1518606400451747
Panoptic Quality (PQ): 0.7192522800512962
volumetric instance-wise DICE: 0.9002926160099541 ± 0.10253566174957332
